# INFORMASI
### Bi-LSTM
### Fine Tuning pada data Twitter, pergunakan weights nya lalu fine-tune kembali pada data YouTube

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional
from sklearn.model_selection import train_test_split

# --- Konfigurasi ---
MAX_NB_WORDS = 5000
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 100

# --- Load Data Twitter ---
df_twitter = pd.read_csv('final_data_twitter.csv')
df_twitter = df_twitter.dropna(subset=['clean_text_ML_2'])

In [2]:
# --- Tokenisasi (Fit pada data Twitter) ---
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(df_twitter['clean_text_ML_2'].values)

X_tw = tokenizer.texts_to_sequences(df_twitter['clean_text_ML_2'].values)
X_tw = pad_sequences(X_tw, maxlen=MAX_SEQUENCE_LENGTH)
y_tw = df_twitter['cyberbullying'].values

# Split Data Twitter
X_temp_tw, X_test_tw, y_temp_tw, y_test_tw = train_test_split(
    X_tw, y_tw, test_size=0.2, random_state=42
)
X_train_tw, X_val_tw, y_train_tw, y_val_tw = train_test_split(
    X_temp_tw, y_temp_tw, test_size=1/8, random_state=42
)

In [3]:
from tensorflow.keras.metrics import Precision, Recall

# --- Custom F1 Metric ---
def f1_metric(y_true, y_pred):
    y_pred = tf.round(y_pred)

    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float32'))
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float32'))
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float32'))

    precision = tp / (tp + fp + 1e-7)
    recall = tp / (tp + fn + 1e-7)

    return 2 * precision * recall / (precision + recall + 1e-7)

# --- Membangun Model Bi-LSTM ---
def create_model():
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        loss='binary_crossentropy',
        optimizer='adam',
        metrics=['accuracy',
                 Precision(name="precision"),
                 Recall(name="recall"),
                 f1_metric
                ]
    )
    return model

In [4]:
# Train Model pada Twitter
model_tw = create_model()
print("Training Model pada Data Twitter...")
model_tw.fit(
    X_train_tw, y_train_tw,
    epochs=8,
    batch_size=64,
    validation_data=(X_val_tw, y_val_tw),
    verbose=1
)

Training Model pada Data Twitter...
Epoch 1/8
143/143 [==============================] - 61s 394ms/step - loss: 0.5371 - accuracy: 0.7261 - precision: 0.7784 - recall: 0.4933 - f1_metric: 0.5229 - val_loss: 0.4234 - val_accuracy: 0.7963 - val_precision: 0.8314 - val_recall: 0.6434 - val_f1_metric: 0.7211
Epoch 2/8
143/143 [==============================] - 49s 343ms/step - loss: 0.3264 - accuracy: 0.8605 - precision: 0.8441 - recall: 0.8222 - f1_metric: 0.8303 - val_loss: 0.3934 - val_accuracy: 0.8263 - val_precision: 0.8232 - val_recall: 0.7445 - val_f1_metric: 0.7789
Epoch 3/8
143/143 [==============================] - 49s 342ms/step - loss: 0.2609 - accuracy: 0.8929 - precision: 0.8795 - recall: 0.8655 - f1_metric: 0.8706 - val_loss: 0.4114 - val_accuracy: 0.8240 - val_precision: 0.8107 - val_recall: 0.7555 - val_f1_metric: 0.7799
Epoch 4/8
143/143 [==============================] - 48s 339ms/step - loss: 0.2172 - accuracy: 0.9113 - precision: 0.8954 - recall: 0.8951 - f1_metric: 0.

In [5]:

model_tw.evaluate(X_test_tw, y_test_tw, verbose=1)

82/82 [==============================] - 3s 37ms/step - loss: 0.6188 - accuracy: 0.8017 - precision: 0.7871 - recall: 0.7301 - f1_metric: 0.7534


[0.6188410520553589,
 0.8016909956932068,
 0.787109375,
 0.7300724387168884,
 0.7534232139587402]

In [6]:
# --- Simpan Weights ---
model_tw.save_weights('bilstm_twitter_weights.h5')
print("Bobot model Twitter berhasil disimpan.")

Bobot model Twitter berhasil disimpan.


In [7]:
# --- Load Data YouTube ---
df_yt = pd.read_csv('final_data_yt.csv')
df_yt = df_yt.dropna(subset=['clean_text_ML_2'])

# --- Preprocessing YouTube ---
X_yt = tokenizer.texts_to_sequences(df_yt['clean_text_ML_2'].values)
X_yt = pad_sequences(X_yt, maxlen=MAX_SEQUENCE_LENGTH)
y_yt = df_yt['cyberbullying'].values

# Split Data YouTube
X_temp_yt, X_test_yt, y_temp_yt, y_test_yt = train_test_split(
    X_yt, y_yt, test_size=0.2, random_state=42
)
X_train_yt, X_val_yt, y_train_yt, y_val_yt = train_test_split(
    X_temp_yt, y_temp_yt, test_size=1/8, random_state=42
)


In [8]:
# --- Load Model & Weights untuk Fine-Tuning ---
model_finetune = create_model()

# Build model sebelum load weights
dummy_input = np.zeros((1, MAX_SEQUENCE_LENGTH))
model_finetune(dummy_input)

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.4977548]], dtype=float32)>

In [9]:

# Load bobot dari Twitter
model_finetune.load_weights('bilstm_twitter_weights.h5')
print("\nBobot Twitter dimuat. Mulai Fine-tuning pada Data YouTube...")


Bobot Twitter dimuat. Mulai Fine-tuning pada Data YouTube...


In [10]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# --- Fine-Tuning ---
model_finetune.fit(
    X_train_yt, y_train_yt,
    epochs=8,
    batch_size=64,
    validation_data=(X_val_yt, y_val_yt),
    callbacks=[es],
    verbose=1
)

Epoch 1/8
8/8 [==============================] - 6s 399ms/step - loss: 1.1951 - accuracy: 0.6176 - precision: 0.5281 - recall: 0.2626 - f1_metric: 0.3061 - val_loss: 0.8352 - val_accuracy: 0.6154 - val_precision: 0.4583 - val_recall: 0.4783 - val_f1_metric: 0.2340
Epoch 2/8
8/8 [==============================] - 3s 341ms/step - loss: 0.7905 - accuracy: 0.6681 - precision: 0.6000 - recall: 0.4693 - f1_metric: 0.4662 - val_loss: 0.7449 - val_accuracy: 0.6308 - val_precision: 0.4815 - val_recall: 0.5652 - val_f1_metric: 0.2600
Epoch 3/8
8/8 [==============================] - 3s 330ms/step - loss: 0.6634 - accuracy: 0.6857 - precision: 0.6139 - recall: 0.5419 - f1_metric: 0.5821 - val_loss: 0.6913 - val_accuracy: 0.6462 - val_precision: 0.5000 - val_recall: 0.5217 - val_f1_metric: 0.2553
Epoch 4/8
8/8 [==============================] - 3s 332ms/step - loss: 0.5889 - accuracy: 0.7055 - precision: 0.6490 - recall: 0.5475 - f1_metric: 0.6200 - val_loss: 0.6599 - val_accuracy: 0.6923 - val_pre

In [11]:

model_finetune.evaluate(X_test_yt, y_test_yt, verbose=1)

5/5 [==============================] - 0s 38ms/step - loss: 0.6742 - accuracy: 0.6846 - precision: 0.5238 - recall: 0.5116 - f1_metric: 0.4023


[0.6741998791694641,
 0.6846153736114502,
 0.523809552192688,
 0.5116279125213623,
 0.4022623896598816]